In [68]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [95]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data.get('headline', soup.find('h1', class_='content--title').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    return notyet

def get_link_nogenre(start=0):
    notyet = []
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    for dic in data:
        if dic['genre'] == []:
            notyet.append(dic['url'])
    return notyet  

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
easy = pd.read_json('nhkwebeasy.json')

In [4]:
htmls = ['https://www3.nhk.or.jp/news/html/20191117/k10012180511000.html']

In [96]:
htmls = get_link(0); htmls

['https://www3.nhk.or.jp/news/html/20190120/k10011784611000.html',
 'https://www3.nhk.or.jp/news/html/20191010/k10012120321000.html',
 'https://www3.nhk.or.jp/news/html/20191017/k10012135411000.html']

In [111]:
htmls = get_link_nogenre(); htmls

ml/20190916/k10012084571000.html',
 'https://www3.nhk.or.jp/news/html/20190917/k10012084901000.html',
 'https://www3.nhk.or.jp/news/html/20190917/k10012085171000.html',
 'https://www3.nhk.or.jp/news/html/20190917/k10012085691000.html',
 'https://www3.nhk.or.jp/news/html/20190918/k10012086671000.html',
 'https://www3.nhk.or.jp/news/html/20190918/k10012086731000.html',
 'https://www3.nhk.or.jp/news/html/20190918/k10012086891000.html',
 'https://www3.nhk.or.jp/news/html/20190918/k10012087661000.html',
 'https://www3.nhk.or.jp/news/html/20190918/k10012087961000.html',
 'https://www3.nhk.or.jp/news/html/20190919/k10012088561000.html',
 'https://www3.nhk.or.jp/news/html/20190919/k10012088691000.html',
 'https://www3.nhk.or.jp/news/html/20190919/k10012089251000.html',
 'https://www3.nhk.or.jp/news/html/20190919/k10012090321000.html',
 'https://www3.nhk.or.jp/news/html/20190920/k10012090631000.html',
 'https://www3.nhk.or.jp/news/html/20190920/k10012090721000.html',
 'https://www3.nhk.or.jp/ne

# scrape

In [50]:
driver = webdriver.Chrome()

In [112]:
for nhkurl in tqdm.tqdm(htmls[:]):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(30)
    html = driver.page_source.encode('utf-8')
    dic = scrape(html)
    js(dic)










  0%|          | 0/528 [00:00<?, ?it/s]








  0%|          | 1/528 [01:22<12:05:07, 82.56s/it]








  0%|          | 2/528 [02:50<12:16:58, 84.07s/it]








  1%|          | 3/528 [04:21<12:33:33, 86.12s/it]








  1%|          | 4/528 [06:02<13:12:01, 90.69s/it]








  1%|          | 5/528 [07:39<13:26:50, 92.56s/it]








  1%|          | 6/528 [09:01<12:56:59, 89.31s/it]








  1%|▏         | 7/528 [10:34<13:06:00, 90.52s/it]








  2%|▏         | 8/528 [12:17<13:37:46, 94.36s/it]








  2%|▏         | 9/528 [13:34<12:51:37, 89.20s/it]








  2%|▏         | 10/528 [14:59<12:38:40, 87.88s/it]








  2%|▏         | 11/528 [16:47<13:29:27, 93.94s/it]








  2%|▏         | 12/528 [18:08<12:54:17, 90.03s/it]








  2%|▏         | 13/528 [19:51<13:25:59, 93.90s/it]








  3%|▎         | 14/528 [21:40<14:02:20, 98.33s/it]

AttributeError: 'NoneType' object has no attribute 'text'

In [94]:
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
dic = scrape(html)
print(dic)
js(dic)

{'id': 'k10011846961000', 'title': '買い物弱者など解決へ ロボットがコンビニの商品配達する実験', 'article': '高齢化に伴う「買い物弱者」などの解決につなげようと、自動で動くロボットがコンビニの商品を配達する実験が神奈川県の大学で行われました。\nこれは大手コンビニチェーンの「ローソン」と自動運転技術を開発するベンチャー企業の「ＺＭＰ」などが行いました。実験は大学内で行われ、利用者がスマートフォンの専用のアプリを使って商品を注文すると、高さ、幅などが１メートルに満たない箱型のロボットが、仮設の店舗から商品を届けます。センサーやカメラが周りの状況を認識しているということで、人が近づくと自動的に停止するほか、人とすれ違った際には音声であいさつもしてくれます。両社では、こうしたロボットで高齢化に伴う買い物弱者やドライバーの人手不足といった課題の解決につなげたい考えです。また公道を移動できるよう国に対して法整備を働きかけていくことにしています。ローソンの牧野国嗣オープン・イノベーションセンター長は「時間がなかったり、体が不自由で店に行けなかったりするなど、さまざまなニーズに応えるため、実用化を進めていきたい」と話していました。', 'genre': ['ビジネス', '社会'], 'keywords': [], 'url': 'https://www3.nhk.or.jp/news/html/20190313/k10011846961000.html', 'datePublished': '2019-03-13T17:35:18+09:00', 'dateModified': '2019-03-27T14:00:23+09:00'}


In [91]:
html = str(driver.page_source.encode('utf-8'))
re.search(r'"genre":(.+?),', html)

# clean category & keyword

In [75]:
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print(len(data))

4712


In [103]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print(len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

4714


[('社会', 1287),
 ('国際', 1075),
 ('ビジネス', 642),
 ('スポーツ', 542),
 ('科学・文化', 537),
 ('政治', 527),
 ('暮らし', 319),
 ('地域', 252),
 ('気象・災害', 124)]

In [102]:
# genre <> keywords

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        j = re.sub('災害|気象', '気象・災害', j)
        if j in category:
            newgenre.append(j)
        else:
            newkey.append(j)
    for j in dic['keywords']:
        j = re.sub('災害|気象', '気象・災害', j)
        if j in category:
            newgenre.append(j)
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    